# ***Read and write files in mongodb***

sudo systemctl status mongod

In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from striprtf.striprtf import rtf_to_text
from pymongo import MongoClient
from pprint import pprint

In [2]:
MAIN_PATH = Path(os.getcwd())
##
def get_text_sentence_raw(sentence):
    path_to_read = MAIN_PATH / "data" / "downloads_files_sentences" / f'{sentence}.rtf'
    with open(path_to_read, 'r') as doc:
        rtf_text = doc.read()
    string_sentence = rtf_to_text(rtf_text)
    return string_sentence

## ***Test write in mongodb***

In [3]:
# DROP mongodb #
#client = MongoClient('localhost', 27017)
# db_name = 'sentences'
# db = client[db_name]
# client.drop_database(db_name)
# client.close()

In [3]:
def write_sentence_in_mongodb(sentence):
    client = MongoClient('localhost', 27017)
    db = client['sentences']
    collection = db['sentences_raw']
    sentence_raw = get_text_sentence_raw(sentence)
    document = {'id_sentence': sentence, 'text_raw': sentence_raw,}
    collection.insert_one(document)
    client.close()
    return None

write_sentence_in_mongodb("C-010-95")

In [4]:
def get_info_mongodb(sentence):
    client = MongoClient('localhost', 27017)
    db = client['sentences']
    collection = db['sentences_raw']
    query = {'id_sentence': sentence}
    document = collection.find_one(query)
    document.pop('_id',None)
    df_result = pd.DataFrame([document])
    return df_result

test = get_info_mongodb("C-010-95")
test

,id_sentence,text_raw
0,C-010-95,Sentencia No. C-010/95\n\nSALARIO INTEGRAL-Exe...


In [5]:
df = pd.read_csv(MAIN_PATH / "data" / "Sentencias_Corte_1992-2022.csv")
df.merge(test, left_on=['Sentencia'], right_on=['id_sentence'], how='inner')

,Sentencia,Proceso,Expediente Tipo,Expediente NÃºmero,Magistrado(a) ponente,Sentencia Tipo,Fecha Sentencia,Año,SV-SPV,AV-APV,id_sentence,text_raw
0,C-010-95,Demanda de inconstitucionalidad,D,634,Jorge Arango MejÃ­a,C,1/17/1995 0:00,1995,No,No,C-010-95,Sentencia No. C-010/95\n\nSALARIO INTEGRAL-Exe...
